In [3]:
from sage.all import *
import math
import secrets
import hashlib
import numpy as np
import pickle

# Algoritmos de apoyo: Parte 1

- Generación de semilla "aleatoria"
- Expansión de semilla a matriz/vector en grupo 
- Expansión de semilla privada
- Árbol de semillas

In [4]:
def generate_seed(bit_size):
    byte_size = bit_size // 8
    return secrets.token_bytes(byte_size)

def expand_seed_to_matrix(seed, n, k, p):
    shake = hashlib.shake_128(seed)
    total_elements = (n - k) * k  # Número total de elementos en la matriz aleatoria
    random_bytes = shake.digest(total_elements)  # Generar los bytes necesarios

    # Convertir los bytes en una matriz de enteros en el rango [0, 255]
    random_values = np.frombuffer(random_bytes, dtype=np.uint8).reshape(n - k, k)%p

    # Crear la matriz identidad de tamaño k x k
    identity_matrix = np.eye((n-k), dtype=np.uint8)

    # Concatenar la matriz aleatoria con la matriz identidad
    final_matrix = np.hstack((identity_matrix, random_values))

    return final_matrix

def expand_seed_to_Group(seed: bytes, group:str, p: int = 7, n: int = 7, t=7, w=4, E: list = [1, 2, 4], Fz: list = [0, 1, 2]):
    # Expandir la semilla con SHAKE128 para generar n bytes adicionales
    shake = hashlib.shake_128()
    shake.update(seed)
    
    # Generar suficientes bytes (uno por elemento del vector)
    expanded_bytes = shake.digest(n)  # Genera 'n' bytes
    
    # Convertir bytes a enteros y mapear a un grupo
    # Grupo Fp (grupo finito con p elementos)
    if group == "Fp":
        vector_fp = vector(GF(p), [b % p for b in expanded_bytes])
        return vector_fp
    # Grupo Fp* (multiplicativo, de 1 a p-1)
    elif group == "Fp*":
        expanded_bytes = shake.digest(t)
        vector_fpm = vector([(b % (p-1)) + 1 for b in expanded_bytes]) 
        return vector_fpm
    # Grupo B (vectores binarios de longitud t y peso w)
    elif group == "B":
        if w > t:
            raise ValueError("El peso w no puede ser mayor que la longitud t")
        
        expanded_bytes = shake.digest(2 * t)  # Más bytes para asegurar distribución uniforme
        indices = sorted(set(b % t for b in expanded_bytes))[:w]  # Seleccionar `w` índices únicos
        
        #Crear el vector binario con 'w' 1s en posiciones aleatorias
        vector_B = [1 if i in indices else 0 for i in range(t)]
        return vector_B 
    elif group == "Fz": 
        vector_E = vector([Fz[b % len(Fz)] for b in expanded_bytes])
        return vector_E
    else: 
        vector_E = vector([E[b % len(E)] for b in expanded_bytes])
        return vector_E

def expand_private_seed(seed_sk, params):
    l, n, k, p = params
    shake = hashlib.shake_128(seed_sk)
    derived_bytes = shake.digest(2*l)
    seed_e, seed_pk = derived_bytes[:l], derived_bytes[l//2:]
    H = expand_seed_to_matrix(seed_pk, n, k, p)
    eta = expand_seed_to_Group(seed_e, 'Fz')
    return eta, H

def seed_tree_leaves(m_seed, salt, params):
    l, t = params
    lambda_bytes = l // 8  # Convertir bits a bytes
    salt_bytes = salt  # Salt de 2λ bytes ya en formato bytes

    # 1. Crear la raíz del árbol
    root = m_seed + salt_bytes  # Raíz = Mseed || Salt
    
    # Lista para almacenar todos los nodos generados en orden
    nodes = {0: root}

    # 2. Construcción del árbol binario
    index = 1  # Para numerar los nodos
    while len(nodes) < t+1:  # Hasta tener suficientes nodos para al menos t hojas
        parent_index = (index - 1) // 2  # Índice del padre en el árbol
        parent_seed = nodes[parent_index]  # Semilla del padre

        # Generar 3λ bits (384 bits si λ=128) como salida del CSPRNG (SHAKE128)
        shake = hashlib.shake_128(parent_seed + salt_bytes)
        pseudo_random_bits = shake.digest(3 * lambda_bytes)  # 3λ bytes

        # Concatenar el índice del nodo (como bytes)
        node_seed = pseudo_random_bits + index.to_bytes(4, 'big')  # 4 bytes para el índice
        nodes[index] = node_seed
        index += 1

    # 3. Extraer las primeras t hojas del árbol
    #leaf_start = len(nodes) // 2  # Primer índice de una hoja
    #leaves = [nodes[i] for i in range(leaf_start, leaf_start + t)]
    seed_tree = [nodes[i] for i in range(1, t+1)]

    return seed_tree

# Algoritmos de apoyo: Parte 2
- MerkleRoot
- MerkleProofs
- RecomputeMerkleRoot

In [5]:
def sha256(data):
    return hashlib.sha256(data).digest()

#Prioridad hacia derecha, árboles balanceados a la izquierda
def setupTree(t):
    off = [0]*int(math.ceil(math.log2(t))+1)
    num = []
    levels = math.ceil(math.log2(t))+1
    for i in range(levels):
        num.append(2**i)
    num[-1] = 2*(t%2**(i-1))
    res_nodes = t - 2*(t%2**(i-1))

    off[-2] = res_nodes
    off[-1] = 2**(i-1) - res_nodes
    return off, num

def leafIndices(off,t):
    idx = []
    nodes_i = 2**(math.ceil(math.log2(t))-1)-1
    nodes_last = 2**(math.ceil(math.log2(t))-1) - ((t%2**(math.floor(math.log2(t)))))
    nodes_count = 0
    off_lvl = 0
    for i in range(t):
        if nodes_count == nodes_last:
            off_lvl = off[-1]
        idx.append(nodes_i + off_lvl)
        nodes_i += 1
        nodes_count += 1
    return idx

def updateCtr(ctr, lvl, num):
    lvl_nodes = num[lvl]
    ctr += 2
    if lvl_nodes == ctr:
        lvl -= 1
        ctr = 0
    return ctr, lvl

def merkleRoot(cmt0):
    t = len(cmt0)
    off, num = setupTree(t)
    idx = leafIndices(off, t)

    T = [None]*(2*t-1)
    for i in range(0, t):
        T[idx[i]] = cmt0[i]
    
    lvl = math.floor(math.log2(t)) + 1
    ctr = 0
    for i in range(2*t-2, 0, -2):
        p = math.floor((i-1)/2) + off[lvl-1]
        T[p] = bytes.fromhex(hashlib.sha3_256(T[i-1] + T[i]).hexdigest())
        ctr, lvl = updateCtr(ctr, lvl, num)
    return T[0], T

def merkleProofs(T, cmt0, b):
    t = len(cmt0)
    off, num = setupTree(t)
    idx = leafIndices(off, t)

    T_ = [0]*(2*t-1)
    for i in range(0, t):
        if b[i] == 0:
            T_[idx[i]] = 1

    lvl = math.ceil(math.log2(t))-1
    ctr = 0
    merkle_proofs = []
    for i in range(2*t-2, 0, -2):
        p = math.floor((i-1)/2) + off[lvl]
        T_[p] = T_[i-1] or T_[i]

        #right child computed, left child not so add it to proof
        if T_[i] == 1 and T_[i-1] == 0:
            merkle_proofs.append(T[i-1])
        
        #left child computed, right child not so add it to proof
        if T_[i] == 0 and T_[i-1] == 1:
            merkle_proofs.append(T[i])
        
        ctr, lvl = updateCtr(ctr, lvl, num)
    
    return merkle_proofs

def recomputeMerkleRoot(cmt0, merkle_proofs, b):
    t = len(cmt0)
    off, num = setupTree(t)
    idx = leafIndices(off, t)

    T_ = [0]*(2*t-1)
    T = [0]*(2*t-1)
    for i in range(0, t):
        if b[i] == 0:
            T_[idx[i]] = 1
            T[idx[i]] = cmt0[i]

    lvl = math.ceil(math.log2(t))-1
    ctr = 0
    l = 0
    h = ['T[i-1]', 'T[i]']
    for i in range(2*t-2, 0, -2):
        if T_[i] == 0 and T_[i-1] == 0:
            ctr, lvl = updateCtr(ctr, lvl, num)
            continue
    
        #take first node from tree if valid, else from proof
        if T_[i] == 1:
            h[1] = T[i]
        else:
            h[1] = merkle_proofs[l]
            l += 1
        
        #take second node from tree if valid, else from proof
        if T_[i-1] == 1:
            h[0] = T[i-1]
        else:
            h[0] = merkle_proofs[l]
            l += 1
        
        p = math.floor((i-1)/2) + off[lvl]
        T[p] = bytes.fromhex(hashlib.sha3_256(h[0]+h[1]).hexdigest())
        T_[p] = 1
        ctr, lvl = updateCtr(ctr, lvl, num)

    d_0 = T[0]
    return d_0

In [6]:
def keyGen(l, n, k, p=7, g=4):
    seed_sk = generate_seed(l)
    shake = hashlib.shake_128(seed_sk)
    derived_bytes = shake.digest(2*l)
    seed_e, seed_pk = derived_bytes[:l], derived_bytes[l//2:]
    H = expand_seed_to_matrix(seed_pk, n, k, p)
    eta = expand_seed_to_Group(seed_e, 'Fz')
    e = vector([g**eta_i for eta_i in eta])
    s = np.matmul(e,H.transpose())%p
    sk = seed_sk
    pk = seed_pk, s
    return sk, pk

sk, pk = keyGen(128, 7, 4)
print(f'sk=>\nseed_sk: {sk}\npk=>\nseed_pk: {pk[0]}\ns: {pk[1]}')

sk=>
seed_sk: b'\x99\xfe\x81\x0b\x14\x918w\xe0L\x80\x87&\xa0c\x14'
pk=>
seed_pk: b'\xc5mR\xc0\xd3o/\x0e\xdf5h( \'H\xfc\xe2>\xb1U6\xda\x84\xcd\x0edd\xc14\xf3\x0e%,)\xeb#NP\'\xff\x88\xe2\xbb\xf2\xba\xf0\xd3\xed\x84\x8c\xdfi\x05\xca\'\xed\x88\x90\'\xedr5\xf3v\x9e\x9d!\x19\x84\xb0KdO\xa3\xc0"\xf4\x03M7\t-\x83\xe3\xec\x10\xbf!\xdcpv\xe4C\xff(\x15\xd0;\x80$\xdd\'7\xa5\xdbc\x97`I>\xe0q\xe8\x8aTL\x1c%\xad\x0e\xdc\xb7\xbe}u\xe1\xc1\xbe\xb7\xab\xed\x9e\x82\xb4i:\xabR\xce}\xc2@\xc6~\xc1\x97\xac\xcd\xb3U}\x05\xb6v\xdd\x1a\xdeE\x9c\x14\xc4\xf0\x0f\xc3\xaf\xcb\xcb\x13\x8e\x1cq\xcc\xc1\n\xe3\xfe[E\x8f\x04\xe9A\xe0p\xdaF\x9f6[p0\xe4'
s: [6 1 0]


In [ ]:
def sign(sk, msg, params):
    l, g, t, n, k, p, Fz = params

    eta, H = expand_private_seed(sk, [l, n, k, p])

    MSeed = generate_seed(l)
    Salt = generate_seed(2*l)
    seeds = seed_tree_leaves(MSeed, Salt, [l, t])
    seedue = []
    eta_list = []
    sigmas = []
    e_s = []
    cmt0, cmt1 = [], []
    for i in range(0, t):
        shake = hashlib.shake_128(seeds[i])
        derived_bytes = shake.digest(2*l)
        seed_u, seed_e = derived_bytes[:l], derived_bytes[l//2:]
        seedue.append(seed_u.hex() + "|" + seed_e.hex())
        eta_ = expand_seed_to_Group(seed_e, 'Fz')
        eta_list.append(eta_)
        sigma_i = eta - eta_
        sigma_i = vector([Fz[b % len(Fz)] for b in sigma_i])
        sigmas.append(sigma_i)
        v = vector([g**sigma_v for sigma_v in sigma_i])%7
        u_ = expand_seed_to_Group(seed_u, 'Fp')
        u = np.multiply(v,u_)%7
        s_ = np.matmul(u, H.transpose())%p
        cmt0_i = bytes([int(x) for x in s_]) + bytes([int(x) for x in sigma_i]) + Salt + i.to_bytes(4, 'big')
        cmt0.append(bytes.fromhex(hashlib.sha3_256(cmt0_i).hexdigest()))
        cmt1_i = seeds[i] + Salt + i.to_bytes(4, 'big')
        cmt1.append(hashlib.sha3_256(cmt1_i).hexdigest())
    d0, T = merkleRoot(cmt0)
    d1 = bytes.fromhex(hashlib.sha3_256(bytes.fromhex(''.join(cmt1))).hexdigest())
    d01 = d0 + d1

    #Extracción del primer vector desafío, en ZKP hecho por el verificador, acá simulado autónomamente
    dm = bytes.fromhex(hashlib.sha3_256(bytes.fromhex(msg.encode('utf-8').hex())).hexdigest())
    dbeta = bytes.fromhex(hashlib.sha3_256(dm + d01 + Salt).hexdigest())
    beta = expand_seed_to_Group(dbeta, 'Fp*')

    #Cálculo del primer round de respuestas 
    y = []
    for i in range(0, t):
        e_ = vector([g**eta_i for eta_i in eta_list[i]])%p
        e_s.append(e_)
        y.append(u_ + beta[i]*e_)
    
    #Extracción del segundo vector desafío
    y_hex = ["".join(bytes([int(x) for x in y_i]).hex()) for y_i in y]
    db = bytes.fromhex(hashlib.sha3_256(bytes.fromhex("".join(y_hex))).hexdigest()) #+ dbeta).hexdigest())
    b = expand_seed_to_Group(db, 'B')

    #Cálculo del segundo round de respuestas
    merkle_proofs = merkleProofs(T, cmt0, b)
    #seed_path = seed_tree_paths(MSeed, b)

    rsp0, rsp1 = [], []
    for i in range(0, t):
        if b[i] == 0:
            rsp0.append((y[i], sigmas[i]))
            rsp1.append(cmt1[i])
    
    signature = MSeed, Salt, d01, db, merkle_proofs, rsp0, rsp1 #, seed_path

    print(f'0. e = {vector([g**eta_i for eta_i in eta])%7}\n                    {H[0]}\n   E = [1,2,4], H = {H[1]}, s = {np.matmul(vector([g**eta_i for eta_i in eta])%7, H.transpose())%7}\n                    {H[0]}')
    print(f'1. MSeed = {MSeed.hex()}\n   Salt = {Salt.hex()}')
    print(f'2. SeedTree = {[seed.hex() for seed in seeds]}')
    print(f"3. Seedu' + Seede' = {seedue}\n   eta = {eta_list}\n   cmt0 = {[cmt0_i.hex() for cmt0_i in cmt0]}\n   cmt1 = {cmt1}")
    print(f'4. d0, T = {d0.hex()}, {[x.hex() for x in T]}')
    print(f'5. d01 = {d01.hex()}\n   dm = {dm.hex()}\n   dbeta = {dbeta.hex()}\n   beta = {beta}')
    print(f"6. e'= {e_s}\n   y = {y}")
    print(f'7. h = {y_hex}\n   db = {db.hex()}\n   b = {vector(b)}')
    print(f'8. Pendiente (por ahora)')
    print(f'9. merkleProofs = {[proof.hex() for proof in merkle_proofs]}')
    print(f'10. rsp0 = {rsp0}\n    rsp1 = {rsp1}')
    print(f'11. Signature = {Salt.hex()} | {d01.hex()} | {db.hex()} | {[proof.hex() for proof in merkle_proofs]} | {rsp0} | {rsp1}')
    
    signature = pickle.dumps(signature)  # Serializa la tupla a bytes
    return signature.hex()

signature = sign(sk, 'Hola Mundo!', [128, 4, 7, 7, 4, 7, [0,1,2]])  
print(signature)  

0. e = (1, 1, 1, 4, 4, 2, 4)
                    [1 0 0 0 5 2 4]
   E = [1,2,4], H = [0 1 0 4 6 5 5], s = [6 1 0]
                    [1 0 0 0 5 2 4]
1. MSeed = 1d1a32b29b73943cb87b37ae2826164a
   Salt = 4ad929a0b7039abb9354867bb83d11f58aff56a9fae5f00e71c47bb9ffaa1ebc
2. SeedTree = ['29fb9e8275bd922480aca695c92218b7f48ed1350ac4501726e1ffeeb71a612fc92f9f84b75fd76723b3551abcb2c86f00000001', '29fb9e8275bd922480aca695c92218b7f48ed1350ac4501726e1ffeeb71a612fc92f9f84b75fd76723b3551abcb2c86f00000002', 'dc9026505fd1f6c593306f0160513c868c5abadf7c85da69ea1a6e20a21c374bec2ddc5821bd5a53c2062f47abc9085500000003', 'dc9026505fd1f6c593306f0160513c868c5abadf7c85da69ea1a6e20a21c374bec2ddc5821bd5a53c2062f47abc9085500000004', '562c4c292677482aab3ae8b5a80313acc200082f6016ef1edf24668a293d727651f14bb36669567a10f3efafdc00ebcf00000005', '562c4c292677482aab3ae8b5a80313acc200082f6016ef1edf24668a293d727651f14bb36669567a10f3efafdc00ebcf00000006', 'c394728ee9b9b8c2637f77816949df61e55d7c102466e8beda708e8aab12eece048

In [22]:
def verify(pk, msg, signature, params):
    l, g, t, n, k, p, Fz = params
    seed_pk, s = pk    
    signature = pickle.loads(bytes.fromhex(signature))

    MSeed, Salt, d01, db, merkle_proofs, rsp0, rsp1 = signature #, seed_path

    H = expand_seed_to_matrix(seed_pk, n, k, p)
    
    #Se recalculan los desafíos
    dm = bytes.fromhex(hashlib.sha3_256(bytes.fromhex(msg.encode('utf-8').hex())).hexdigest())
    dbeta = bytes.fromhex(hashlib.sha3_256(dm + d01 + Salt).hexdigest())
    beta = expand_seed_to_Group(dbeta, 'Fp*')
    b = expand_seed_to_Group(db, 'B')

    #seeds = rebuildSeedTreeLeaves(seed_path, b, Salt)
    seeds = seed_tree_leaves(MSeed, Salt, [l, t])

    j = 0
    cmt0 = []
    cmt1 = []
    y = []
    for i in range(0, t):
        if b[i] == 1:
            cmt1_i = seeds[i] + Salt + i.to_bytes(4, 'big')
            cmt1.append(hashlib.sha3_256(cmt1_i).hexdigest())
            shake = hashlib.shake_128(seeds[i])
            derived_bytes = shake.digest(2*l)
            seed_u, seed_e = derived_bytes[:l], derived_bytes[l//2:]
            eta_ = expand_seed_to_Group(seed_e, 'Fz')
            e_ = vector([g**eta_i for eta_i in eta_])%p
            u_ = expand_seed_to_Group(seed_u, 'Fp')
            y.append(u_ + beta[i]*e_)
        else:
            y_i, sigma_i = rsp0[j]
            y.append(y_i)
            v = vector([g**sigma_v for sigma_v in sigma_i])%7
            y_ = np.multiply(v,y_i)%7
            s_ = np.matmul(y_, H.transpose())%p - beta[i]*s
            cmt0_i = bytes([int(x) for x in s_]) + bytes([int(x) for x in sigma_i]) + Salt + i.to_bytes(4, 'big')
            cmt0.append(bytes.fromhex(hashlib.sha3_256(cmt0_i).hexdigest()))
            cmt1.append(rsp1[j])
            j += 1
    d_0 = recomputeMerkleRoot(cmt0, merkle_proofs, b)
    d_1 = bytes.fromhex(hashlib.sha3_256(bytes.fromhex(''.join(cmt1))).hexdigest())
    d_01 = d_0 + d_1
    y_hex = ["".join(bytes([int(x) for x in y_i]).hex()) for y_i in y]
    d_b = bytes.fromhex(hashlib.sha3_256(bytes.fromhex("".join(y_hex))).hexdigest())
    
    print(f'                      {H[0]}\n0.   E = [1,2,4], H = {H[1]}, s = {s}\n                      {H[0]}')
    print(f'1. beta = {beta}')
    print(f'2. b = {b}')
    print(f"3. Pendiente (por ahora)")
    print(f"4-5. cmt0 = {[cmt0_i.hex() for cmt0_i in cmt0]}\n   cmt1 = {cmt1}\n   y = {y}")
    print(f"6. d0'= {d_0.hex()}\n   d1 = {d_1.hex()}")
    print(f"7-8. d'01'= {d_01.hex()}\n   d'b = {d_b.hex()}")

    if d01 == d_01 and db == d_b:
        return True
    else:
        return False
    
verify(pk, 'Hola Mundo!', signature, [128, 4, 7, 7, 4, 7, [0,1,2]])

                      [1 0 0 0 5 2 4]
0.   E = [1,2,4], H = [0 1 0 4 6 5 5], s = [6 1 0]
                      [1 0 0 0 5 2 4]
1. beta = (3, 4, 1, 2, 6, 3, 6)
2. b = [1, 0, 1, 1, 0, 1, 0]
3. Pendiente (por ahora)
4-5. cmt0 = ['58f41c61a863f6b1c51f9e8dde8a7fb220b770c31492f06d3346c66ffabc0e06', '8f0ca7bdc096093ddb3ff5ba1025d7752818d88cf2f0dd9ff6b3d1c53f1c4ab7', '3acfe27e8446e2db279cd08326765cfdca5a3630256f976de162f21386e2da1b']
   cmt1 = ['69bbe8bfb98ee2ed4ce62f5ff3d76bea9a86fc38a4cdd88fa0761e2f6978003c', '4d0b72ac89441412828c603dfbc769ed0abb30999d5407fcde0bb5d8d40c94e1', '0dc3e4da79efec240b1c418c1077a92d6562f884e48403c38ac5a608f5d80b6e', '2e47c4265e430137838058116edf1c0d212e748df112420c2eedfbcd092d4662', 'f5e21bfd51d08f2cc3e16244cb7a6f12a15dfcd6681e7efe1bf40a2bcb731981', '4052907bbdd925577a47476a1549ef43235c4a400dd61808f0aed75a44d81464', '81f7c027a6e032e948b82ec2d131aebc455611eb5c4e11ecf2d833f822581c53']
   y = [(0, 6, 6, 3, 5, 2, 0), (6, 6, 3, 2, 4, 0, 2), (4, 3, 3, 0, 2, 0, 1), (5, 1,

False